In [1]:
import numpy as np
import math
import sys
sys.path.append('../../python/')  
from periphery import constant
from periphery import logicGate
print(constant.INV)

0
0


In [2]:
class Technology:
    def __init__(self, node_nm=45, roadmap='HP', transistor_type='conventional'):
        self.node_nm = node_nm
        self.roadmap = roadmap
        self.transistor_type = transistor_type
        self.initialized = False
        self.params = {}
        self._initialize()

    def _initialize(self):
        if self.initialized:
            print("Warning: Already initialized!")
            return
        if self.transistor_type == 'conventional':
            if self.node_nm == 45 and self.roadmap == 'HP':
                vdd = 1.0
                vth = 0.18
                phyGateLength = 45e-9
                capIdealGate = 4e-10
                capFringe = 5e-10
                capOverlap = capIdealGate * 0.2 if self.node_nm >= 22 else 0.0

                # Junction cap model (from BSIM4)
                buildInPotential = 0.9
                cjd = 1e-3
                cjswd = 2.5e-10
                cjswgd = 0.5e-10
                mjd = 0.5
                mjswd = 0.33
                mjswgd = 0.33
                #/* Properties not used so far */
                capPolywire = 0.0;	#/* TO-DO: we need to find the values */

                capJunction = cjd / pow(1 + vdd / buildInPotential, mjd)
                capSidewall = cjswd / pow(1 + vdd / buildInPotential, mjswd)
                capDrainToChannel = cjswgd / pow(1 + vdd / buildInPotential, mjswgd)

                # Current arrays (μA/μm)
                currentOnNmos = [930e-6 - i * 2e-6 for i in range(0, 101, 10)]      # NMOS on-current
                currentOnPmos = [700e-6 - i * 1.5e-6 for i in range(0, 101, 10)]    # PMOS on-current
                currentOffNmos = [100e-9 + i * 0.5e-9 for i in range(0, 101, 10)]   # NMOS off-current
                currentOffPmos = [80e-9 + i * 0.4e-9 for i in range(0, 101, 10)]    # PMOS off-current

                # Interpolation to full 0-100
                currentOnNmos = self._interpolate_full(currentOnNmos)
                currentOnPmos = self._interpolate_full(currentOnPmos)
                currentOffNmos = self._interpolate_full(currentOffNmos)
                currentOffPmos = self._interpolate_full(currentOffPmos)

                self.params = {
                    'vdd': vdd,
                    'vth': vth,
                    'phyGateLength': phyGateLength,
                    'capIdealGate': capIdealGate,
                    'capFringe': capFringe,
                    'capOverlap': capOverlap,
                    'capJunction': capJunction,
                    'capSidewall': capSidewall,
                    'capDrainToChannel': capDrainToChannel,
                    'effectiveResistanceMultiplier': 1.0,  # could be adjusted based on design
                    'current_gmNmos': 1e-3,  # conductance for gm unit is μS/μm
                    'current_gmPmos': 0.8e-3,
                    'currentOnNmos': currentOnNmos,
                    'currentOnPmos': currentOnPmos,
                    'currentOffNmos': currentOffNmos,
                    'currentOffPmos': currentOffPmos,
                    'pnSizeRatio': 2.0,
                    'transistorType': self.transistor_type,   
                    'capPolywire': capPolywire,  
                    'featureSize': self.node_nm * 1e-9       
                }

            elif self.node_nm == 14 and self.roadmap == 'LSTP':
                vdd = 0.8
                vth = 0.45
                phyGateLength = 14e-9
                capIdealGate = 2.5e-10
                capFringe = 3e-10
                capOverlap = 0.0  # don't use overlap cap in finFET nodes

                buildInPotential = 0.9
                cjd = 1e-3
                cjswd = 2.5e-10
                cjswgd = 0.5e-10
                mjd = 0.5
                mjswd = 0.33
                mjswgd = 0.33
                #/* Properties not used so far */
                capPolywire = 0.0;	#/* TO-DO: we need to find the values */

                capJunction = cjd / pow(1 + vdd / buildInPotential, mjd)
                capSidewall = cjswd / pow(1 + vdd / buildInPotential, mjswd)
                capDrainToChannel = cjswgd / pow(1 + vdd / buildInPotential, mjswgd)

                currentOnNmos = [600e-6 - i * 2e-6 for i in range(0, 101, 10)]
                currentOnPmos = [500e-6 - i * 1.5e-6 for i in range(0, 101, 10)]
                currentOffNmos = [10e-9 + i * 0.1e-9 for i in range(0, 101, 10)]
                currentOffPmos = [8e-9 + i * 0.1e-9 for i in range(0, 101, 10)]

                currentOnNmos = self._interpolate_full(currentOnNmos)
                currentOnPmos = self._interpolate_full(currentOnPmos)
                currentOffNmos = self._interpolate_full(currentOffNmos)
                currentOffPmos = self._interpolate_full(currentOffPmos)

                self.params = {
                    'vdd': vdd,
                    'vth': vth,
                    'phyGateLength': phyGateLength,
                    'capIdealGate': capIdealGate,
                    'capFringe': capFringe,
                    'capOverlap': capOverlap,
                    'capJunction': capJunction,
                    'capSidewall': capSidewall,
                    'capDrainToChannel': capDrainToChannel,
                    'effectiveResistanceMultiplier': 1.2,
                    'current_gmNmos': 2.0e-3,
                    'current_gmPmos': 1.6e-3,
                    'currentOnNmos': currentOnNmos,
                    'currentOnPmos': currentOnPmos,
                    'currentOffNmos': currentOffNmos,
                    'currentOffPmos': currentOffPmos,
                    'capPolywire': capPolywire,  
                    'pnSizeRatio': 2.0
                }
            else:
                raise ValueError(f"Unsupported node {self.node_nm}nm or roadmap {self.roadmap}")

            self.initialized = True

    def _interpolate_full(self, base):
        'gemerate a full 0-100 array from a base array with 10 steps'
        full = [0.0] * 101
        for i in range(0, 101, 10):
            full[i] = base[i // 10]
        for i in range(1, 100):
            if i % 10 != 0:
                low = (i // 10) * 10
                high = low + 10
                alpha = (i - low) / 10
                full[i] = full[low] * (1 - alpha) + full[high] * alpha
        return full

    def get_param(self, name):
        return self.params.get(name, None)

    def interpolate_current(self, name, bias_index):
        'Interpolate current values based on bias index (0-100)'
        if name not in self.params:
            return None
        arr = self.params[name]
        i_low = int(bias_index)
        i_high = min(100, i_low + 1)
        alpha = bias_index - i_low
        return arr[i_low] * (1 - alpha) + arr[i_high] * alpha

    def print_summary(self):
        print(f"Tech Node: {self.node_nm}nm, Roadmap: {self.roadmap}")
        for k, v in self.params.items():
            if isinstance(v, list):
                print(f"{k}: {v[:3]} ... {v[-3:]}")
            else:
                print(f"{k}: {v:.3e}" if isinstance(v, float) else f"{k}: {v}")


In [36]:
config = {
    'mode': 'memory',
    'technology': '65',
    'device_type': 'SRAM',
    'frequency': 1e9,            # 1 GHz
    'precision_mu': 6,        # Device variation mean
    'precision_sigma': 2,    # Device variation stddev
    'dataset': 'CIFAR-10',
    'temperature': 350,  # Kelvin
    'model': 'ResNet18',
    'distribution_file': "../../../DATA/customized_gaussian_current.csv"
}

In [46]:
class RowDecoder:
    def __init__(self, mode, num_addr_row, mux, parallel, tech, config):
        self.tech = tech
        self.config = config
        self.initialized = False
        self.initialize(mode, num_addr_row, mux, parallel)

    def initialize(self, mode, num_addr_row, mux, parallel):
        self.mode = mode
        self.numAddrRow = num_addr_row
        self.MUX = mux
        self.parallel = parallel
        self.featureSize = self.tech.get_param('featureSize')
        self.pnSizeRatio = self.tech.get_param('pnSizeRatio')

        scale = 8 if parallel else 1

        # use 2 bit predecoder for 2^n address lines
        # INV
        self.width_inv_n = scale * constant.MIN_NMOS_SIZE * self.featureSize
        self.width_inv_p = scale * self.pnSizeRatio * constant.MIN_NMOS_SIZE * self.featureSize
        self.num_inv = num_addr_row # the INV at output driver stage does not count here

        # NAND2
        self.width_nand_n = scale * 2 * constant.MIN_NMOS_SIZE  * self.featureSize
        self.width_nand_p = scale * self.pnSizeRatio * constant.MIN_NMOS_SIZE * self.featureSize
        self.num_nand = 4 * int(math.floor(num_addr_row / 2))

        # NOR2
        self.width_nor_n = scale * constant.MIN_NMOS_SIZE * self.featureSize
        self.num_input_nor = int(math.ceil(num_addr_row / 2))
        self.width_nor_p = scale * self.num_input_nor * self.pnSizeRatio * constant.MIN_NMOS_SIZE * self.featureSize
        self.num_nor = int(2 ** num_addr_row) if num_addr_row > 2 else 0

        self.num_metal_connection = self.num_nand + (num_addr_row % 2) * 2 if num_addr_row > 2 else 0

        self.width_driver_inv_n = scale * 3 * constant.MIN_NMOS_SIZE * self.featureSize
        self.width_driver_inv_p = scale * 3 * self.pnSizeRatio * constant.MIN_NMOS_SIZE * self.featureSize

        self.initialized = True

    def calculate_area(self, max_transistor_height, new_height, new_width, option, constant, m_pitch):
        if not self.initialized:
            raise RuntimeError("RowDecoder must be initialized before area calculation.")

        # Magic layout options
        MAGIC, OVERRIDE, NONE = 'MAGIC', 'OVERRIDE', 'NONE'

        # Gate areas
        h_inv, w_inv, _ = logicGate.calculate_logicgate_area(constant.INV,1, self.width_inv_n , self.width_inv_p, self.featureSize * max_transistor_height, self.tech)
        h_nand, w_nand, _ = logicGate.calculate_logicgate_area(constant.NAND,2, self.width_nand_n, self.width_nand_p, self.featureSize * max_transistor_height, self.tech)
        h_nor, w_nor, _ = logicGate.calculate_logicgate_area(constant.NOR,self.num_input_nor, self.width_nor_n , self.width_nor_p, self.featureSize * max_transistor_height, self.tech)
        h_driver_inv, w_driver_inv, _ = logicGate.calculate_logicgate_area(constant.INV, 1, self.width_driver_inv_n , self.width_driver_inv_p, self.featureSize * max_transistor_height, self.tech)

        if self.mode == "REGULAR_ROW":
            if new_height and option == NONE:
                if max(h_inv, h_nand, h_nor) > new_height:
                    raise ValueError("RowDecoder cell height exceeds assigned layout height.")

                # NOR
                num_nor_per_col = int(new_height / h_nor)
                num_nor_per_col = min(num_nor_per_col, self.num_nor)
                num_col_nor = math.ceil(self.num_nor / num_nor_per_col) if num_nor_per_col > 0 else 0

                # NAND
                num_nand_per_col = int(new_height / h_nand)
                num_nand_per_col = min(num_nand_per_col, self.num_nand)
                num_col_nand = math.ceil(self.num_nand / num_nand_per_col) if num_nand_per_col > 0 else 0

                # INV
                num_inv_per_col = int(new_height / h_inv)
                num_inv_per_col = min(num_inv_per_col, self.num_inv)
                num_col_inv = math.ceil(self.num_inv / num_inv_per_col)

                height = new_height
                width = (
                    w_inv * num_col_inv +
                    w_nand * num_col_nand +
                    m_pitch * self.num_metal_connection * self.featureSize +
                    w_nor * num_col_nor
                )
                if self.MUX:
                    width += (w_nand + w_inv * 2) * num_col_nor
                else:
                    width += w_driver_inv * 2 * num_col_nor
            else:
                height = max(h_nor * self.num_nor, h_nand * self.num_nand)
                width = (
                    w_inv + w_nand +
                    m_pitch * self.num_metal_connection * self.featureSize +
                    w_nor
                )
                if self.MUX:
                    width += w_nand + w_inv * 2
                else:
                    width += w_driver_inv * 2

        elif self.mode == "REGULAR_COL":
            if new_width and option == NONE:
                if max(w_inv, w_nand, w_nor) > new_width:
                    raise ValueError("RowDecoder cell width exceeds assigned layout width.")

                # NOR
                num_nor_per_row = int(new_width / w_nor)
                num_nor_per_row = min(num_nor_per_row, self.num_nor)
                num_row_nor = math.ceil(self.num_nor / num_nor_per_row) if num_nor_per_row > 0 else 0

                # NAND
                num_nand_per_row = int(new_width / w_nand)
                num_nand_per_row = min(num_nand_per_row, self.num_nand)
                num_row_nand = math.ceil(self.num_nand / num_nand_per_row) if num_nand_per_row > 0 else 0

                # INV
                num_inv_per_row = int(new_width / w_inv)
                num_inv_per_row = min(num_inv_per_row, self.num_inv)
                num_row_inv = math.ceil(self.num_inv / num_inv_per_row)

                width = new_width
                height = (
                    h_inv * num_row_inv +
                    h_nand * num_row_nand +
                    m_pitch * self.num_metal_connection * self.featureSize +
                    h_nor * num_row_nor
                )
                if self.MUX:
                    height += (h_nand + h_inv * 2) * num_row_nor
                else:
                    height += h_driver_inv * 2 * num_row_nor
            else:
                height = (
                    h_inv + h_nand +
                    m_pitch * self.num_metal_connection * self.featureSize +
                    h_nor
                )
                width = max(w_nor * self.num_nor, w_nand * self.num_nand)
                if self.MUX:
                    height += h_nand + h_inv * 2
                else:
                    height += h_driver_inv * 2

        else:
            raise ValueError("Unsupported mode for RowDecoder.")

        area = height * width

        # capacitance
        # inv
        self.cap_inv_input, self.cap_inv_output = logicGate.calculate_logicgate_cap(constant.INV, 1, self.width_inv_n, self.width_inv_p, h_inv, self.tech)
        # nand
        if self.num_nand > 0:
            self.cap_nand_input, self.cap_nand_output = logicGate.calculate_logicgate_cap(constant.NAND, 2, self.width_nand_n, self.width_nand_p, h_nand, self.tech)
        else:
            self.cap_nand_input, self.cap_nand_output = 0, 0
        # nor
        if self.num_nor > 0:
            self.cap_nor_input, self.cap_nor_output = logicGate.calculate_logicgate_cap(constant.NOR, self.num_input_nor, self.width_nor_n, self.width_nor_p, h_nor, self.tech)
        else:
            self.cap_nor_input, self.cap_nor_output = 0, 0

        # driver inv
        self.cap_driver_inv_input, self.cap_driver_inv_output = logicGate.calculate_logicgate_cap(constant.INV, 1, self.width_driver_inv_n, self.width_driver_inv_p, h_driver_inv, self.tech)
        return {"height": height, "width": width, "area": area}

    def calculate_latency(self, ramp_input, cap_load1, cap_load2, num_read, num_write):
        if not self.initialized:
            raise RuntimeError("RowDecoder must be initialized before latency calculation.")

        read_latency = 0
        write_latency = 0

        ramp_inv_output = 1e20
        ramp_nand_output = 1e20
        ramp_nor_output = 1e20
        ramp_output = 1e20

        # --- INV stage ---
        res_pull_down = logicGate.calculate_on_resistance(self.width_inv_n, constant.NMOS, self.config['temperature'], self.tech)
        if self.num_nand:
            tr = res_pull_down * (self.cap_inv_output + self.cap_nand_input * 2) # one address line connects to 2 NAND inputs
        else:
            tr = res_pull_down * (self.cap_inv_output + cap_load1)
        gm = logicGate.calculate_transconductance(self.width_inv_n, constant.NMOS, self.tech)
        beta = 1 / (res_pull_down * gm)
        print(f"RowDecoder: INV tr={tr}, gm={gm}, beta={beta}")
        read_latency += logicGate.horowitz(tr, beta, ramp_input)[0]
        write_latency += logicGate.horowitz(tr, beta, ramp_input)[0]
        print(f"RowDecoder: INV read_latency={read_latency}, write_latency={write_latency}")
        if not self.num_nand:
            ramp_output = ramp_inv_output

        # --- NAND stage ---
        if self.num_nand:
            res_pull_down = logicGate.calculate_on_resistance(self.width_nand_n, constant.NMOS, self.config['temperature'], self.tech) * 2
            if self.num_nor:
                tr = res_pull_down * (self.cap_nand_output + self.cap_nor_input * self.num_nor / 4)
            else:
                tr = res_pull_down * (self.cap_nand_output + cap_load1)
            gm = logicGate.calculate_transconductance(self.width_nand_n, constant.NMOS, self.tech)
            beta = 1 / (res_pull_down * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_inv_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_inv_output)[0]
            if not self.num_nor:
                ramp_output = ramp_nand_output

        # --- NOR stage ---
        if self.num_nor:
            res_pull_up = logicGate.calculate_on_resistance(self.width_nor_p, constant.PMOS, self.config['temperature'], self.tech) * 2
            if self.MUX:
                tr = res_pull_up * (self.cap_nor_output + self.cap_nand_input)
            else:
                tr = res_pull_up * (self.cap_nor_output + self.cap_inv_input)
            gm = logicGate.calculate_transconductance(self.width_nor_p, constant.PMOS, self.tech)
            beta = 1 / (res_pull_up * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_nand_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_nand_output)[0]
            ramp_output = ramp_nor_output

        # --- Output driver or MUX ---
        if self.MUX:
            # NAND
            res_pull_down = logicGate.calculate_on_resistance(self.width_nand_n, constant.NMOS, self.config['temperature'], self.tech)
            tr = res_pull_down * (self.cap_nand_output + self.cap_inv_input)
            gm = logicGate.calculate_transconductance(self.width_nand_n, constant.NMOS, self.tech)
            beta = 1 / (res_pull_down * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_output)[0]

            # 1st INV
            res_pull_up = logicGate.calculate_on_resistance(self.width_inv_p, constant.PMOS, self.config['temperature'], self.tech)
            tr = res_pull_up * (self.cap_inv_output + self.cap_inv_input + cap_load1)
            gm = logicGate.calculate_transconductance(self.width_inv_p, constant.PMOS, self.tech)
            beta = 1 / (res_pull_up * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_nand_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_nand_output)[0]

            # 2nd INV
            res_pull_down = logicGate.calculate_on_resistance(self.width_inv_n, constant.NMOS, self.config['temperature'], self.tech)
            tr = res_pull_down * (self.cap_inv_output + cap_load2)
            gm = logicGate.calculate_transconductance(self.width_inv_n, constant.NMOS, self.tech)
            beta = 1 / (res_pull_down * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_inv_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_inv_output)[0]

        else:
            # REGULAR: 2 INV as output driver
            res_pull_down = logicGate.calculate_on_resistance(self.width_driver_inv_n, constant.NMOS, self.config['temperature'], self.tech)
            tr = res_pull_down * (self.cap_driver_inv_output + self.cap_driver_inv_input)
            gm = logicGate.calculate_transconductance(self.width_driver_inv_n, constant.NMOS, self.tech)
            beta = 1 / (res_pull_down * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_output)[0]

            res_pull_up = logicGate.calculate_on_resistance(self.width_driver_inv_p, constant.PMOS, self.config['temperature'], self.tech)
            tr = res_pull_up * (self.cap_driver_inv_output + cap_load1)
            gm = logicGate.calculate_transconductance(self.width_driver_inv_p, constant.PMOS, self.tech)
            beta = 1 / (res_pull_up * gm)
            read_latency += logicGate.horowitz(tr, beta, ramp_inv_output)[0]
            write_latency += logicGate.horowitz(tr, beta, ramp_inv_output)[0]

        read_latency *= num_read
        write_latency *= num_write

        return {"read_latency": read_latency, "write_latency": write_latency}
    

    def calculate_power(self, num_read, num_write):
        if not self.initialized:
            raise RuntimeError("RowDecoder must be initialized before power calculation.")

        self.leakage = 0
        self.read_dynamic_energy = 0
        self.write_dynamic_energy = 0
        vdd = self.tech['vdd']

        # Leakage
        # INV
        self.leakage += logicGate.calculate_gate_leakage(
            constant.INV, 1, self.width_inv_n, self.width_inv_p, self.tech['temperature'], self.tech) * vdd * self.num_inv

        self.leakage += logicGate.calculate_gate_leakage(
            constant.NAND, 2, self.width_nand_n, self.width_nand_p, self.tech['temperature'], self.tech) * vdd * self.num_nand

        self.leakage += logicGate.calculate_gate_leakage(
            constant.NOR, self.num_input_nor, self.width_nor_n, self.width_nor_p, self.tech['temperature'], self.tech) * vdd * self.num_nor

        # Output driver or MUX selector
        if self.MUX:
            self.leakage += logicGate.calculate_gate_leakage(
                constant.NAND, 2, self.width_nand_n, self.width_nand_p, self.tech['temperature'], self.tech) * vdd * self.num_nor
            self.leakage += logicGate.calculate_gate_leakage(
                constant.INV, 1, self.width_inv_n, self.width_inv_p, self.tech['temperature'], self.tech) * vdd * 2 * self.num_nor
        else:
            self.leakage += logicGate.calculate_gate_leakage(
                constant.INV, 1, self.width_driver_inv_n, self.width_driver_inv_p, self.tech['temperature'], self.tech) * vdd * 2 * self.num_nor

        # Dynamic energy calculation
        floor_half = int(self.numAddrRow // 2)
        read_factor = vdd ** 2

        # --- Read energy ---
        # INV stage
        self.read_dynamic_energy += (
            (self.cap_inv_input + self.cap_nand_input * 2) * read_factor * floor_half * 2
        )
        self.read_dynamic_energy += (
            (self.cap_inv_input + self.cap_nor_input * (self.num_nor / 2)) * read_factor * (self.numAddrRow - floor_half * 2)
        )
        # NAND stage
        self.read_dynamic_energy += (
            (self.cap_nand_output + self.cap_nor_input * self.num_nor / 4) * read_factor * self.num_nand / 4
        )

        # NOR stage
        if self.MUX:
            self.read_dynamic_energy += (self.cap_nor_output + self.cap_nand_input) * read_factor   #one NOR output activated
        else:
            self.read_dynamic_energy += (self.cap_nor_output + self.cap_inv_input) * read_factor    #one NOR output activated
        # Output driver or MUX selector
        if self.MUX:
            self.read_dynamic_energy += (self.cap_nand_output + self.cap_inv_input) * read_factor
            self.read_dynamic_energy += (self.cap_inv_output + self.cap_inv_input) * read_factor
            self.read_dynamic_energy += self.cap_inv_output * read_factor
        else:
            self.read_dynamic_energy += (self.cap_driver_inv_input + self.cap_driver_inv_output) * read_factor * 2

        # --- Write energy (same pattern) ---
        # INV stage
        self.write_dynamic_energy += (
            (self.cap_inv_input + self.cap_nand_input * 2) * read_factor * floor_half * 2
        )
        self.write_dynamic_energy += (
            (self.cap_inv_input + self.cap_nor_input * (self.num_nor / 2)) * read_factor * (self.numAddrRow - floor_half * 2)
        )
        # NAND stage
        self.write_dynamic_energy += (
            (self.cap_nand_output + self.cap_nor_input * self.num_nor / 4) * read_factor * self.num_nand / 4
        )
        # NOR stage
        if self.MUX:
            self.write_dynamic_energy += (self.cap_nor_output + self.cap_nand_input) * read_factor
            # Output driver or MUX selector
            self.write_dynamic_energy += (self.cap_nand_output + self.cap_inv_input) * read_factor
            self.write_dynamic_energy += (self.cap_inv_output + self.cap_inv_input) * read_factor
            self.write_dynamic_energy += self.cap_inv_output * read_factor
        else:
            self.write_dynamic_energy += (self.cap_nor_output + self.cap_inv_input) * read_factor
            self.write_dynamic_energy += (self.cap_driver_inv_input + self.cap_driver_inv_output) * read_factor * 2

        # Scale by access counts
        self.read_dynamic_energy *= num_read
        self.write_dynamic_energy *= num_write


In [47]:
tech45 = Technology(node_nm=45, roadmap='HP')
row_decoder = RowDecoder(
    mode="REGULAR_ROW",
    num_addr_row=4,
    mux=False,
    parallel=False,
    tech=tech45,
    config=config
)

In [48]:
area_result = row_decoder.calculate_area(
    max_transistor_height=constant.MAX_TRANSISTOR_HEIGHT,
    new_height=0,
    new_width=0,
    option='NONE',
    constant=constant,
    m_pitch=2
)
print("Area Result:", area_result)

maxtransistor height: 1.2600000000000002e-06
maxwidth_PMOS: 6.270000000000001e-07
maxwidth_NMOS: 3.135000000000001e-07
num_Folded_PMOS: 1
maxtransistor height: 1.2600000000000002e-06
maxwidth_PMOS: 4.702500000000001e-07
maxwidth_NMOS: 4.702500000000001e-07
num_Folded_PMOS: 1
maxtransistor height: 1.2600000000000002e-06
maxwidth_PMOS: 7.524000000000002e-07
maxwidth_NMOS: 1.881e-07
num_Folded_PMOS: 1
maxtransistor height: 1.2600000000000002e-06
maxwidth_PMOS: 6.270000000000001e-07
maxwidth_NMOS: 3.135000000000001e-07
num_Folded_PMOS: 1
Area Result: {'height': 8.208e-06, 'width': 7.020000000000001e-06, 'area': 5.762016000000001e-11}


In [49]:
latency_result = row_decoder.calculate_latency(
    ramp_input=0.1e-9,
    cap_load1=5e-15,
    cap_load2=5e-15,
    num_read=1,
    num_write=1
)
print("Latency Result:", latency_result)

RowDecoder: INV tr=1.8985059368727896e-05, gm=3.461538461538462e-10, beta=0.21580000000000002
RowDecoder: INV read_latency=308.0994917938678, write_latency=308.0994917938678
Latency Result: {'read_latency': 308.09953722892084, 'write_latency': 308.09953722892084}


In [ ]:






row_decoder.calculate_power(num_read=1, num_write=1)
print("Power Result:")
print(f"  Leakage: {row_decoder.leakage:.3e} W")
print(f"  Read Dynamic Energy: {row_decoder.read_dynamic_energy:.3e} J")
print(f"  Write Dynamic Energy: {row_decoder.write_dynamic_energy:.3e} J")